# The task

Ok, so let's do some fake news detection.

Given a headline and a body of text, we want to say whether these two are:

* unrelated
* agree with each other
* disagree with each other
* discuss each other

In [1]:
from utils.data import DataSet
from utils.features import *
from utils.dependencies import *
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

training = DataSet()
test = DataSet(name="competition_test")

Using TensorFlow backend.


Reading dataset in dataset/
Loading files train_bodies.csv, train_stances.csv
Reading dataset in dataset/
Loading files competition_test_bodies.csv, competition_test_stances.csv


In [2]:
def baseline_model():
    # create model
    #
    #
    #
    #
    return model

In [3]:
d = DataSet("train")
folds,hold_out = kfold_split(d,n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

# Load the competition dataset
competition_dataset = DataSet("competition_test")
X_competition, y_competition = generate_features(competition_dataset.data, "competition")

training = DataSet()
test = DataSet(name="competition_test")
Xs = dict()
ys = dict()

# Load/Precompute all features now
X_holdout,y_holdout = generate_features(hold_out_stances,"holdout")
for fold in fold_stances:
    Xs[fold],ys[fold] = generate_features(fold_stances[fold],str(fold))

best_score = 0
best_fold = None

encoder = LabelEncoder()

# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]
    
    encoder.fit(y_train)
    encoded_Y = encoder.transform(y_train)
    dummy_y = np_utils.to_categorical(encoded_Y)

    estimator = # 
    estimator.fit(X_train,dummy_y)
    
    predicted = [LABELS[int(a)] for a in estimator.predict(X_test)]
    actual = [LABELS[int(a)] for a in y_test]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score/float(max_fold_score)

    print("Score for fold "+ str(fold) + " was - " + str(score))
    if score > best_score:
        best_score = score
        best_fold = estimator

Reading dataset in dataset/
Loading files train_bodies.csv, train_stances.csv
Reading dataset in dataset/
Loading files competition_test_bodies.csv, competition_test_stances.csv
Reading dataset in dataset/
Loading files train_bodies.csv, train_stances.csv
Reading dataset in dataset/
Loading files competition_test_bodies.csv, competition_test_stances.csv
Score for fold 0 was - 0.8024025496445207
Score for fold 1 was - 0.7927084788378265
Score for fold 2 was - 0.7865658621692353
Score for fold 3 was - 0.8108217514505465
Score for fold 4 was - 0.7828015452854485
Score for fold 5 was - 0.7606774668630338
Score for fold 6 was - 0.7638609594943981
Score for fold 7 was - 0.7690567105077075
Score for fold 8 was - 0.804063492063492
Score for fold 9 was - 0.7789561819704158


In [4]:
#Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
actual = [LABELS[int(a)] for a in y_holdout]

print("Scores on the dev set")
report_score(actual,predicted)
print("")
print("")

#Run on competition dataset
predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,predicted)

Scores on the dev set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    153    |    32     |    497    |    80     |
-------------------------------------------------------------
| disagree  |    20     |    28     |    103    |    11     |
-------------------------------------------------------------
|  discuss  |    161    |    54     |   1384    |    201    |
-------------------------------------------------------------
| unrelated |    20     |     3     |    179    |   6696    |
-------------------------------------------------------------
Score: 3455.75 out of 4448.5	(77.68348881645498%)


Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    326    |    43    

74.01137217036799